In [1]:
from mfst import FST

FST('hello')

FST {
0	1	104	True
1	2	101	True
2	3	108	True
3	4	108	True
4	5	111	True
5	True

}

In [2]:
FST('hello').union(FST('help'))

FST(num_states=11)

In [3]:
_.remove_epsilon().determinize()

FST {
0	1	104	True
1	2	101	True
2	3	108	True
3	4	108	True
3	5	112	True
4	6	111	True
5	True
6	True

}

In [4]:
transducer = FST()
s = transducer.add_state()
transducer.set_initial_state(s)
for letter_from, letter_to in zip('hello', 'world'):
    next_state = transducer.add_state()
    transducer.add_arc(from_state=s, 
                       to_state=next_state, 
                       weight=1,
                       input_label=letter_from,
                       output_label=letter_to)
    s = next_state
transducer.set_final_weight(s, weight=1)
transducer

FST {
0	1	104	119	1
1	2	101	111	1
2	3	108	114	1
3	4	108	108	1
4	5	111	100	1
5	1

}

In [5]:
Out[3].compose(Out[4]).project('output')

FST {
0	1	119	1
1	2	111	1
2	3	114	1
3	4	108	1
4	5	100	1
5	1

}

In [6]:
help(FST().determinize)

Help on method determinize in module mfst:

determinize(delta=0.0009765625, weight_threshold=None, *, allow_non_functional=False) method of mfst.FST instance
    This operation determinizes a weighted transducer. The result will be an
    equivalent FST that has the property that no state has two transitions
    with the same input label. For this algorithm, epsilon transitions are
    treated as regular symbols (cf. RmEpsilon).
    
    http://www.openfst.org/twiki/bin/view/FST/DeterminizeDoc
    
    delta: Quantization delta for binning weights.
    weight_threshold: Pruning weight threshold.
    allow_non_functional: Only works on path semirings.  In the case that there are two
      output sequences for a given input the one with the shorter path
      will be used and the longer path discarded.



In [7]:
from mfst import RealSemiringWeight
weighted = FST(RealSemiringWeight)
start = weighted.add_state()
weighted.set_initial_state(start)
def build_transduction(from_word, to_word):
    state = start
    for letter_from, letter_to in zip(from_word, to_word):
        next_state = weighted.add_state()
        weight = 2 if letter_to == 'l' else 1
        weighted.add_arc(from_state=state, to_state=next_state,
                         weight=weight,
                         input_label=letter_from, output_label=letter_to)
        state = next_state
    weighted.set_final_weight(state, weight=3)
build_transduction('hello', 'world')
build_transduction('hello', 'troll')
weighted

FST(num_states=11)

In [8]:
import torch
from mfst import AbstractSemiringWeight

add_layer = torch.nn.Linear(20, 10)
mul_layer = torch.nn.Linear(20, 10)

# Note: not a "real" semiring
class TorchSemiring(AbstractSemiringWeight):
    def __init__(self, value):
        self.value = value
    def __add__(self, other):
        inp = torch.cat((self.value, other.value))
        out = torch.sigmoid(add_layer(inp))  # pytorch forward
        return TorchSemiring(out)
    def __mul__(self, other):
        inp = torch.cat((self.value, other.value))
        out = torch.sigmoid(mul_layer(inp))  # pytorch forward
        return TorchSemiring(out)
    def __str__(self): return f'TW({self.value[0]:.2f})'
    def __eq__(s, o): return all(s.value == o.value)
    def __hash__(self): return hash(self.value)
    def approx_eq(s, o, delta): 
        return sum(abs(s.value - o.value)) < delta
    
    
TorchSemiring.zero = \
   TorchSemiring(torch.rand(10, requires_grad=True))
TorchSemiring.one = \
   TorchSemiring(torch.rand(10, requires_grad=True))

[weight1, weight2, weight3, weight4] = \
   [TorchSemiring(torch.rand(10, requires_grad=True))
    for _ in range(4)]

torch_fst = FST(TorchSemiring)
[a,b,c] = [torch_fst.add_state() for _ in range(3)]
torch_fst.set_initial_state(a)
torch_fst.add_arc(from_state=a,to_state=b,weight=weight1,
                  input_label='a', output_label='b')
torch_fst.add_arc(from_state=b,to_state=c,weight=weight2,
                  input_label='c', output_label='d')
torch_fst.add_arc(from_state=a,to_state=c,weight=weight3,
                  input_label='e', output_label='f')
torch_fst.set_final_weight(c, weight=weight4)
torch_fst

FST {
0	1	97	98	TW(0.35)
0	2	101	102	TW(0.12)
1	2	99	100	TW(0.08)
2	TW(0.12)

}

In [9]:
FST('e').compose(torch_fst)

FST {
0	1	101	102	TW(0.43)
1	TW(0.42)

}

In [10]:
_.sum_paths()

TorchSemiring(TW(0.44))

In [11]:
# get the resulting pytorch tensor from the FST
numerator = FST('e').compose(torch_fst).sum_paths().value
denominator = torch_fst.sum_paths().value

prob = torch.exp(torch.sum(numerator)) \
     / torch.exp(torch.sum(denominator))
prob.backward()  # pytorch auto diff
## Here I would run the optimizer, IF I HAD ONE
# optimizer.step()
prob

tensor(0.9848, grad_fn=<DivBackward0>)

In [12]:
from mfst import RealSemiringWeight
fst = FST(RealSemiringWeight)
s = fst.add_state()
read_a = fst.add_state()
fst.set_initial_state(s)
fst.set_final_weight(s, weight=1)
fst.add_arc(from_state=s,to_state=s, 
    input_label='a', output_label='a', weight=1)
fst.add_arc(from_state=s,to_state=read_a, 
    # 0 represents epsilon (nothing) read or written
    input_label='a', output_label=0, weight=1)
fst.add_arc(from_state=read_a,to_state=s, 
    input_label='a', output_label='b', weight=.5)
fst.add_arc(from_state=s,to_state=s, 
    input_label='b', output_label='b', weight=1)

fst

FST {
0	0	97	97	1
0	1	97	0	1
0	0	98	98	1
0	1
1	0	97	98	0.5

}

In [13]:
FST('aaa').compose(fst)

FST {
0	1	97	0	1
0	2	97	97	1
1	3	97	98	0.5
2	4	97	0	1
2	3	97	97	1
3	5	97	97	1
4	5	97	98	0.5
5	1

}

In [14]:
from mfst import MinPlusSemiringWeight
FST('aaa').compose(fst).project('output') \
  .lift(MinPlusSemiringWeight, lambda w: w.value) \
  .shortest_path()

FST {
3	2	0	1
0	1
1	0	97	1
2	1	98	0.5

}

In [15]:
from mfst import MaxPlusSemiringWeight
FST('aaa').compose(fst).project('output') \
  .lift(MaxPlusSemiringWeight, lambda w: w.value) \
  .shortest_path()

FST {
3	2	97	1
0	1
1	0	97	1
2	1	97	1

}

In [16]:
from mfst import MinPlusSemiringWeight
min_fst = \
FST('aaa').compose(fst) \
  .lift(MinPlusSemiringWeight, lambda w: w.value) \
  .shortest_path()
next(min_fst.iterate_paths())

Path(input_path='aaa', output_path='ba', weight=MinPlusSemiringWeight(3.5))

In [17]:
class ChangeWidth:
    def _repr_html_(self):
        return '<style> .container { width: 920px !important; }</style>'
ChangeWidth()